# Написание нейронных сетей на PyTorch

В этом ноутбуке мы разберем как писать нейронные сети на фреймворке PyTorch

## Полносвязная нейронная сеть

Полносвязная нейронная сеть - простейший вид нейронных сетей. Но не смотря на это, такие сети применяются очень часто. В полносвязной нейронной сети каждый нейрон одного слоя связан с каждым нейроном последующего слоя. Пример можно увидеть на изображении:

![полносвязная нейронная сеть](img/FC_NN.png)

На этом изображении показана нейронная сеть, состоящая из одного входного слоя (зеленый), двух скрытых слоев (синий) и одного выходного слоя (красный).

Давайте реализуем такую архитектуру на фреймворке PyTorch. PyTorch - это внешняя библиотека по этому для ее установки нужно использовать команду `pip install pytorch`

In [9]:
import torch.nn as nn # Импортируем модуль nn, в котором содержется функционал для написания нейронных сетей
import torch.nn.functional as F # Импортируем модуль functional, в котором содержатся полезные функции для работы с сетками

# Каждая нейронная сеть представляет из себя класс, который наследуется от класса nn.Module
class FullyConnected(nn.Module):
    # В любой нейронной сети должен быть конструктор
    def __init__(self):
        # В конструкоре обязательно нужно вызвать конструктор базового класса nn.Module
        super(FullyConnected, self).__init__()
        
        # Далее идет описание слоев сети
        # Полносвязный слой описывается с помощью класса nn.Linear.
        # Первым параметром конструктора является размерность входящих в него векторов 
        # (т.е. cколько нейронов стояло на предыдущем слое)
        # Вторым параметром конструктора является количество нейронов данного слоя 
        self.hidden1 = nn.Linear(4, 4) # Описываем первый скрытый слой (на вход ему поступает вектор размерности 4; в слое 4 нейрона)
        self.hidden2 = nn.Linear(4, 3) # Описываем второй скрытый слой (на вход ему поступает вектор размерности 4; в слое 3 нейрона)
        self.out = nn.Linear(3, 2) # Описываем выходной слой (на вход ему поступает вектор размерности 3; в слое 2 нейрона)
    
    # В функции forward описывается логика работы сети, т.е. как информация от одного слоя пердается к другому
    # x - входные данные для сети. x - матрица, размер которой - (размер_батча, размер_входных_данных)
    def forward(self, x):
        # Посылаем входные данные в первый скрытый слой.
        # После слоя добавляем функцию активации ReLU.
        # На выходе получается матрица размером (размер_батча, 4)
        x = F.relu(self.hidden1(x))
        # Посылаем входные данные в второй скрытый слой.
        # После слоя добавляем функцию активации ReLU.
        # На выходе получается матрица размером (размер_батча, 3)
        x = F.relu(self.hidden2(x))
        # Посылаем входные данные в выходной слой.
        # После слоя добавляем функцию активации Sigmoid.
        # На выходе получается матрица размером (размер_батча, 2)
        x = F.sigmoid(self.out(x))
        return x

# Создаем экземпляр класса нейронной сети
fc_nn = FullyConnected()
# Выводим на экран архитектуру нашей сети
print(fc_nn)

FullyConnected(
  (hidden1): Linear(in_features=4, out_features=4, bias=True)
  (hidden2): Linear(in_features=4, out_features=3, bias=True)
  (out): Linear(in_features=3, out_features=2, bias=True)
)


Теперь обучим нашу сеть. Для начала создадим входную и выходную переменные - наши обучающие данные 

In [15]:
import torch # импортируем pytorch
# импортируем класс-обертку Variable, который позволяет пропускать градиенты через переменные
from torch.autograd import Variable 

# Для начала создадим тензор размерности 64 х 4 - входные данные, заполненный случайными вещественными числами
# Оборачиваем созданный тензор в обертку Variable 
# Указываем, что вычисления будут проходить на CPU
x = Variable(torch.randn((64, 4))).to('cpu')
# Создадим тензор размерности 64 - выходные лейблы для задачи классификации, заполненный случайными целыми числами 0 или 1
labels = Variable(torch.randint(0, 1, (64,))).to('cpu')

# Выведем размерности тензоров
print(x.shape)
print(labels.shape)

torch.Size([64, 4])
torch.Size([64])


Зададим гиперпараметры нашей сети

In [33]:
N_EPOCHS = 1000 # Количество эпох обучения
LEARNING_RATE = 0.1 # Коэффициент скорости обучения
PRINT_EVERY = 50 # Через сколько эпох нужно выводить результаты на экран

Укажем функцию ошибки и алгоритм обучения сети

In [17]:
import torch.optim as optim # Импортируем модуль optim, в котором содержатся классы-оптимизаторы сетей

# В качестве функции ошибки выберем кросс-энтропию, которая используется в задачах классификации
criterion = nn.CrossEntropyLoss()
# В качестве алгоритма обучения будем использовать стохастический градиентный спуск
# Первым параметром для конструктора класса являются веса сети, которые нужно оптимизировать
# Кроме того, передаем в него скорость обучения
optimizer = optim.SGD(fc_nn.parameters(), lr=LEARNING_RATE)

Далее опишем цикл обучения сети

In [35]:
# Цикл, в котором будем перебирать эпохи обучения
for n_epoch in range(1, N_EPOCHS+1):
    # В начале каждой итерации обнуляем градиенты 
    optimizer.zero_grad()
    
    # Посылаем входные данные в сеть и получаем выходные данные, полученные в ходе работы сети
    output = fc_nn(x)
    
    # Для вычисления ошибки используем выбранный нами критерий - кросс-энтропию
    # На вход передаем выходы сети и метки классов - лейблы, чтобы он сравнил то,
    # что сгенерировала сеть с реальными значениями
    loss = criterion(output, labels)
    
    # Вычисляем градиенты по каждому из весов сети
    loss.backward()
    
    # Делаем шаг алгоритма обучения
    optimizer.step()
    
    if n_epoch % PRINT_EVERY == 0:
        print('Epoch: {} | Loss: {:.6f}'.format(n_epoch, loss.item()))

Epoch: 50 | Loss: 0.313386
Epoch: 100 | Loss: 0.313385
Epoch: 150 | Loss: 0.313383
Epoch: 200 | Loss: 0.313382
Epoch: 250 | Loss: 0.313381
Epoch: 300 | Loss: 0.313379
Epoch: 350 | Loss: 0.313378
Epoch: 400 | Loss: 0.313377
Epoch: 450 | Loss: 0.313376
Epoch: 500 | Loss: 0.313375
Epoch: 550 | Loss: 0.313374
Epoch: 600 | Loss: 0.313373
Epoch: 650 | Loss: 0.313371
Epoch: 700 | Loss: 0.313370
Epoch: 750 | Loss: 0.313369
Epoch: 800 | Loss: 0.313368
Epoch: 850 | Loss: 0.313367
Epoch: 900 | Loss: 0.313366
Epoch: 950 | Loss: 0.313365
Epoch: 1000 | Loss: 0.313364


## Упражнения

### Упражнение 1

Как вы уже поняли из примера, данные в сеть поступают по батчам. Каждый батч состоит из заранее заданного числа обучающих примеров (семплов). При этом все семплы должны иметь одну и ту же размерность. При обработке текста возникает проблема: длина текстов может различаться. Однако, в нашем случае тексты статей из Википедии довольно длинные и каждую статью можно разбить на несколько семплов.

В своей работе мы будем использовать особый вид нейронных сетей - языковые модели (Language Model, LM). Суть языковых моделей заключается в том, что они угадываю слово по контексту. В качестве контекста может использоваться несколько слов, стоящих перед угадываемым словом. То есть в нашем случае входными данными для сети будет контекст, а выходными - угадываемое слово.

На вход сети нельзя просто подать слова, нужно подавать индексы слов. По этому давайте воспользуемся нашим словарем с собственной информацией и создадим новый словарь, отображающий слова в индексы.

Для начала создадим функцию, которая будет принимать на вход словарь с собственной информацией и возвращать список пар (слово, собственная информация)

In [ ]:
import pickle
def dict2pairs(word2selfinformation: dict) -> list:

    result = []
    for j in word2selfinformation:
        result.append(j, word2selfinformation[j]))
    return result

# Прочитайте созданный вами в предыдущей лабораторной файл pickle, в котором хранится словарь слово - собственная информация
with open('word2information.pkl', 'rb') as pkl_file:
    word2information = pickle.load(pkl_file)
    
word2information_list = dict2pairs(word2information)
print (word2information_list[0])
print (word2information_list[1])

Используя собственную информацию, мы можем легко отсортировать слова по частоте их встречаемости. Давайте сделаем это (Подсказка: используйте метод .sort( ), который есть у всех списков):

In [ ]:
# Функция, которая принимает на вход список пар (слово, собственная информация)
# и возвращает слова в порядке уменьшения частоты их встречаемости 
def sort_words(pairs_list: list) -> list:
    return sorted(pairs_list,key=lambda word:word[1])

sorted_words = sort_words(word2information_list)
print(sorted_words[0])
print(sorted_words[1])

Когда у нас есть список слов, упорядоченных в порядке уменьшения частоты их встречаемости, мы можем сопоставить каждому слову его индекс:

In [ ]:
import pickle
# Функция, на вход которой поступает отсортированный список слов, а на выходе - словарь "слово-индекс". 
# Индексация должна начинаться с 1.
def word_index(sorted_words: list) -> dict:
    result = {}
    for j in sorted_words:
        result[j[0]] = j[1]
    return result

word2index = word_index(sorted_words)

print(word2index['.'])

# Создадим словарь, для сохранения данных в формате pickle
data_dict = {
    'word2index': word2index,
    'index2word': sorted_words
}

# Сохраним словарь в виде pickle-файла
with open('dictionary.pkl', 'wb') as pkl_out:
    pickle.dump(data_dict, pkl_out, protocol=pickle.HIGHEST_PROTOCOL)

А теперь давайте напишем пару полезных функций, которая понадобятся нам в дальнейшем. Ин ачнем мы с функции, которая превращает текст в набор индексов. 

In [ ]:
# На вход функции подается словарь "слово-индекс" и токенизированный текст (список токенов),
# который мы хотим преобразовать в набор индексов. А на выходе получается список индексов.
# Предусмотрите возможность, что слова может не оказаться в словаре. Для этого воспользуйтесь исключениями (try - except)
def tokens2indexes(tokens: list, word2index: dict) -> list:
    result = []
    for word in tokens:
        try:
            result.append(word2index[word])
        except Exception:
            continue

    return result


tokens = ['i', 'might', 'consider', 'asking', 'for', 'the', 'card', 'to', 'be', 'refunded', '.']

# Прочитайте только что созданный pickle-файл со словарем и возьмите из него словарь word2index
with open('dictionary.pkl', 'rb') as pkl_in:
    word2index = pickle.load(pkl_in)['word2index']

indexes = tokens2indexes(tokens, word2index)

print(indexes)

Следующая функция будет нужна нам для вычисления информации, которую содержит текст. Собственная информация текста равна сумме собственных информаций каждого из токенов текста

In [ ]:
# На вход функции подается словарь "слово-информация" и токенизированный текст (список токенов),
# собственную информацию которого мы хотим найти. А на выходе получается одно единственное вещественное число -
# 
# Предусмотрите возможность, что слова может не оказаться в словаре. Для этого воспользуйтесь исключениями (try - except)
def text2information(tokens: list, word2information: dict) -> float:
    result = 0
    for word in tokens:
        try:
            result+=word2index[word]
        except Exception:
    return result

tokens = ['i', 'might', 'consider', 'asking', 'for', 'the', 'card', 'to', 'be', 'refunded', '.']

# Прочитайте созданный вами в предыдущей лабораторной файл pickle, в котором хранится словарь слово - собственная информация
with open('word2information.pkl', 'rb') as pkl_in:
    word2index = pickle.load(pkl_in)

self_information = text2information(tokens, word2information)

print(self_information)

### Упражнение 2

Теперь давайте применим на практике те знания, которые мы получили в результате разбора примера программирования глубокой нейронной сети на фреймворке PyTorch. 

Вам предлагается реализовать автоенкодер - сеть, которая воссоздает на выходе свои исходные данные. Пример автоенкодера приведен на рисунке:

![полносвязная нейронная сеть](img/autoencoder.png)

Каждый автоенкодер состоит из двух частей - енкодера, сжимающего входные данные, и декодера, восстанавливающего исходные данные. В нашем случае енкодер состоит из трех скрытых слоев, а декодер - из двух скрытых и одного выходного слоя.

Для начала давайте напишем класс, описывающий автоенкодер, приведенный на изображении.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        self.hidden1 = nn.Linear(4,4)
        self.hidden2 = nn.Linear(4,3)
        self.hidden3 = nn.Linear(3,2)
        self.hidden4 = nn.Linear(2,3)
        self.hidden5 = nn.Linear(3,4)
        self.out = nn.Linear(4,4)
    
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = F.relu(self.hidden4(x))
        x = F.relu(self.hidden5(x))
        x = F.relu(self.out(x)
        return x

autoencoder = Autoencoder()
print(autoencoder)

Создадим входные данные для автоенкодера. Поскольку автоенкодер воссоздает исходные данные, роль выходных данных будут играть те же входные данные. В нашем случае сделаем размер батча 256.

In [ ]:
import torch 
from torch.autograd import Variable 

x = Variable(torch.randn((256, 4))).to('gpu')
print(x.shape)

Задайте гиперпараметры сети:

In [ ]:
N_EPOCHS = 1000# Количество эпох обучения
LEARNING_RATE = 0.00001 # Коэффициент скорости обучения
PRINT_EVERY = 50 # Через сколько эпох нужно выводить результаты на экран

Задайте функцию ошибки и алгоритм обучения. В примере у нас была задача классификации. Теперь мы пытаемся воссоздавать исходные данные, при этом значения входных и выходных данных у нас непрерывные. Подумайте, какуюю функцию ошибки нужно применить в этом случае. 

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parametrs(),lr=LEARNING_RATE)

Реализуйте цикл обучения сети

In [ ]:
for n_epoch in range(1, N_EPOCHS+1):
    optimizer.zero_grad()
    output = autoencoder(x)
    loss = criterion(output, x)
    loss.backward()
    optimizer.step()

    if n_epoch % PRINT_EVERY == 0:
        print('Epoch: {} | Loss: {:.6f}'.format(n_epoch, loss.item()))